# Environment setup

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install -q -U  datasets \
                    accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-anookw4b
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-anookw4b
  Resolved https://github.com/huggingface/transformers to commit 0290ec19c901adc0f1230ebdccad11c40af026f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8661022 sha256=aa0b6cfcc87e00f64efd7b035c38c7b2d796d2360e847966240d1ca6c60b9957
  Stored in directory: /tmp/pip-ephem-wheel-cache-9_k7udjo/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

from datasets import load_dataset

In [3]:
# more details on https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0
model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

# Building chatbot

## Using huggingface's chat template from scratch

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [9]:
# Check the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a content creator who is in love with technology that always repond as a geek style.",
    },
    {"role": "user", "content": "How to increase views for my content?"},
]

In [11]:
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True,
                                               add_generation_prompt=True,
                                               return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

<|system|>
You are a content creator who is in love with technology that always repond as a geek style.</s> 
<|user|>
How to increase views for my content?</s> 
<|assistant|>



In [12]:
outputs = model.generate(tokenized_chat, max_new_tokens=128,
                                        do_sample=True,
                                        temperature=0.7,
                                        top_k=50,
                                        top_p=0.95)
print(tokenizer.decode(outputs[0]))

<|system|>
You are a content creator who is in love with technology that always repond as a geek style.</s> 
<|user|>
How to increase views for my content?</s> 
<|assistant|>
Here are some ways to increase views for your content:

1. Create high-quality content: Make sure your content is engaging, informative, and relevant to your target audience.

2. Optimize your content for SEO: Ensure your content is optimized for search engines by including relevant keywords in your title, description, and content.

3. Use social media: Share your content on social media platforms like Twitter, Facebook, and Instagram to reach a wider audience.

4. Promote your content: Leverage social media platforms to promote your content and drive traffic to your website.


## Using huggingface's chat template as pipeline

In [15]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto")

messages = [
    {
        "role": "system",
        "content": "You are a content creator who is in love with technology that always repond as a geek style.",
    },
    {"role": "user", "content": "How to increase views for my content?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=128,
               do_sample=True,
               temperature=0.7,
               top_k=50,
               top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You are a content creator who is in love with technology that always repond as a geek style.</s>
<|user|>
How to increase views for my content?</s>
<|assistant|>
Here are some tips to increase views for your content:

1. Optimize your content: Make sure your content is optimized for search engines. Use descriptive and relevant keywords, use headings and subheadings to break up your content, and add meta descriptions to your articles.

2. Share your content on social media: Share your content on your social media channels, including Facebook, Twitter, LinkedIn, and Instagram. This will help increase visibility and reach your target audience.

3. Use social media advertising: Use social media advertising to reach a wider audience. Platforms like Facebook
